## Performance Overhead

### We use Probabilistic model because :

Simplicity:
It’s computationally light and does not depend on dynamic inputs (like agent distance, signal strength, or topology). This ensures the performance overhead comes purely from the API's wrapper logic, not the model's complexity.

Consistent Timing:
It uses simple random number generation per agent pair. This avoids fluctuations caused by environment-specific variables, which would skew the timing and memory readings.

Baseline Comparability:
Since the only thing changing between baseline and wrapped version is the failure mask logic (not the observation structure or graph computation), the % overhead is meaningful and isolated.

In [1]:
import time
import psutil
import gc
import tracemalloc
import numpy as np
from mpe2 import simple_spread_v3
from pettingzoo.utils.conversions import aec_to_parallel
from failure_api.wrappers import CommunicationWrapper
from failure_api.communication_models import ProbabilisticModel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import multiprocessing as mp
sns.set(style="whitegrid")

# Disable multiprocessing warnings
os.environ["OMP_NUM_THREADS"] = "1"
if hasattr(mp, "set_start_method"):
    try:
        mp.set_start_method("spawn", force=True)
    except RuntimeError:
        pass 


In [2]:

def measure_detailed_performance(env, episodes=10, max_cycles=25):
    agent_ids = env.possible_agents
    step_times = []
    masking_times = []
    gc_counts = []
    fps_list = []
    memory_snapshots = []
    cpu_usages = []

    process = psutil.Process(os.getpid())
    tracemalloc.start()
    gc_old = gc.get_count()

    for _ in range(episodes):
        observations, _ = env.reset()
        for _ in range(max_cycles):
            actions = {
                agent: env.action_space(agent).sample()
                for agent in agent_ids if agent in observations
            }

            # CPU usage before step
            cpu_before = psutil.cpu_percent(interval=None)

            # Start timing full step
            start_step = time.perf_counter()

            # Execute step
            observations, _, _, _, _ = env.step(actions)
            time.sleep(0.1)

            end_step = time.perf_counter()
            cpu_after = psutil.cpu_percent(interval=None)

            step_time = end_step - start_step
            step_times.append(step_time)
            fps_list.append(1.0 / step_time if step_time > 0 else 0)

            # Record CPU
            cpu_usages.append(cpu_after)

            # Memory snapshot
            memory_snapshots.append(process.memory_info().rss / (1024 ** 2))  # in MB

            # GC delta
            gc_counts.append(tuple(np.subtract(gc.get_count(), gc_old)))

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    return {
        "mean_step_time_ms": np.mean(step_times) * 1000,
        "std_step_time_ms": np.std(step_times) * 1000,
        "max_step_time_ms": np.max(step_times) * 1000,
        "min_step_time_ms": np.min(step_times) * 1000,
        "fps": np.mean(fps_list),
        "gc_events": np.sum(gc_counts, axis=0),
        "cpu_usage_mean": np.mean(cpu_usages),
        "cpu_usage_std": np.std(cpu_usages),
        "memory_usage_mean_MB": np.mean(memory_snapshots),
        "memory_usage_std_MB": np.std(memory_snapshots),
        "memory_peak_MB": peak / (1024 ** 2)
        
    }
def run_enhanced_overhead_test(agent_counts=[3, 10, 25], episodes=20, max_cycles=25):
    results = []

    for N in agent_counts:
        print(f"\n🔍 Evaluating {N} agents")

        # Baseline env
        base_env = simple_spread_v3.env(N=N, max_cycles=max_cycles)
        parallel_base = aec_to_parallel(base_env)
        base_metrics = measure_detailed_performance(parallel_base, episodes, max_cycles)

        # Failure_API env
        env = simple_spread_v3.env(N=N, max_cycles=max_cycles)
        agent_ids = env.possible_agents
        failure_model = ProbabilisticModel(agent_ids=agent_ids, failure_prob=0.5)
        wrapped = CommunicationWrapper(env, failure_models=[failure_model])
        parallel_wrapped = aec_to_parallel(wrapped)
        api_metrics = measure_detailed_performance(parallel_wrapped, episodes, max_cycles)

        result = {
            "N": N,
            "baseline_time_ms": base_metrics["mean_step_time_ms"],
            "api_time_ms": api_metrics["mean_step_time_ms"],
            "std_baseline_time": base_metrics["std_step_time_ms"],
            "std_api_time": api_metrics["std_step_time_ms"],
            "% overhead": ((api_metrics["mean_step_time_ms"] - base_metrics["mean_step_time_ms"]) /
                           base_metrics["mean_step_time_ms"]) * 100,
            "baseline_fps": base_metrics["fps"],
            "api_fps": api_metrics["fps"],
            "baseline_cpu": base_metrics["cpu_usage_mean"],
            "api_cpu": api_metrics["cpu_usage_mean"],
            "baseline_mem_MB": base_metrics["memory_usage_mean_MB"],
            "api_mem_MB": api_metrics["memory_usage_mean_MB"],
            "baseline_mem_std": base_metrics["memory_usage_std_MB"],
            "api_mem_std": api_metrics["memory_usage_std_MB"],
            "baseline_gc": base_metrics["gc_events"],
            "api_gc": api_metrics["gc_events"],
            "api_memory_peak_MB": api_metrics["memory_peak_MB"]
        }
        results.append(result)

    df = pd.DataFrame(results)
    print("\n✅ Test Complete. Summary:")
    print(df.to_string(index=False))
    return df

import cProfile
import pstats

def run_profiled_test():
    run_enhanced_overhead_test(agent_counts=[3, 10, 25], episodes=20, max_cycles=25)

# Create a profiler
profiler = cProfile.Profile()

# Run the function under profiler
profiler.enable()
run_profiled_test()
profiler.disable()

# Dump raw profile stats to a file (optional for Snakeviz or dot visualization)
profiler.dump_stats("failure_api_profile.prof")

# Print top 30 cumulative time functions
stats = pstats.Stats(profiler)
stats.strip_dirs().sort_stats("cumtime").print_stats(30)



🔍 Evaluating 3 agents

🔍 Evaluating 10 agents

🔍 Evaluating 25 agents

✅ Test Complete. Summary:
 N  baseline_time_ms  api_time_ms  std_baseline_time  std_api_time  % overhead  baseline_fps  api_fps  baseline_cpu  api_cpu  baseline_mem_MB  api_mem_MB  baseline_mem_std  api_mem_std      baseline_gc         api_gc  api_memory_peak_MB
 3        106.620634   114.496242           3.987345      8.282155    7.386570      9.391421 8.774578       27.3160  24.4180       207.085938  209.475281          0.230031     0.034146 [-34139, 163, 0] [159095, 0, 0]            0.174484
10        137.364680   170.121686          18.110629     31.557654   23.846746      7.392553 6.017978       22.5568  24.6800       209.583211  209.605687          0.011582     0.014472    [27861, 0, 0]  [36174, 0, 0]            0.170699
25        230.686591   471.760018          49.496126     30.188718  104.502575      4.460268 2.127794       24.8668  22.9356       209.844195  210.430109          0.116028     0.124369    [63

In [ ]:
df = run_enhanced_overhead_test()


🔍 Evaluating 3 agents

🔍 Evaluating 10 agents

🔍 Evaluating 25 agents
